In [19]:
import pandas as pd
import ast  # To convert string lists to actual lists

# Load data
access_df = pd.read_csv("Accessibility.csv")
dept_df = pd.read_csv("Department.csv")
emp_df = pd.read_csv("Employees_table.csv")

# Convert Access_ID column from string to list (if it's stored as a string)
emp_df["Access_ID"] = emp_df["Access_ID"].apply(ast.literal_eval)

# Merge Employees with Department
merged_df = emp_df.merge(dept_df, on="Dept_ID", how="left")

# Create a dictionary to map Access_ID to Access name
access_map = dict(zip(access_df["Access_ID"], access_df["Access"]))

# Create a new column with Access Names list using the mapping
merged_df["Access_Name"] = merged_df["Access_ID"].apply(lambda id_list: [access_map.get(i) for i in id_list])

# Final dataframe with all relevant information
final_df = merged_df[["Emp_name", "Dept_ID", "Dept_Name", "Access_ID", "Access_Name"]]

# Display the first few rows
final_df.head()


,Emp_name,Dept_ID,Dept_Name,Access_ID,Access_Name
0,Ajay Chhajlani,1,Accounts and Finance,"[1, 2, 3, 4, 9, 16]","[Teams, Outlook_Internal, Outlook_External, Ou..."
1,Amar Gupta,1,Accounts and Finance,"[1, 2, 3, 4, 9, 16]","[Teams, Outlook_Internal, Outlook_External, Ou..."
2,Amit Bansode,1,Accounts and Finance,"[1, 2, 3, 4, 9, 16]","[Teams, Outlook_Internal, Outlook_External, Ou..."
3,Aniket Khode,1,Accounts and Finance,"[1, 2, 3, 4, 9, 16]","[Teams, Outlook_Internal, Outlook_External, Ou..."
4,Kajal Prajapati,1,Accounts and Finance,"[1, 2, 3, 4, 9, 16]","[Teams, Outlook_Internal, Outlook_External, Ou..."


In [20]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Convert each row to a natural language sentence
def row_to_text(row):
    access_list = ", ".join(row["Access_Name"])
    return (f"{row['Emp_name']} works in the {row['Dept_Name']} department. "
            f"They have access to {access_list}.")

# Apply to final_df
final_df["qa_text"] = final_df.apply(row_to_text, axis=1)


C:\Users\adity\AppData\Local\Temp\ipykernel_25660\3291300696.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["qa_text"] = final_df.apply(row_to_text, axis=1)


In [21]:
# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Create embeddings
final_df["embedding"] = final_df["qa_text"].apply(lambda x: model.encode(x))


c:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\adity\AppData\Local\Temp\ipykernel_25660\2540488078.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["embedding"] = final_df["qa_text"].apply(lambda x: model.encode(x))


In [22]:
def answer_query(query, df, top_k=1):
    # Encode query
    query_vec = model.encode(query)

    # Compute cosine similarities
    similarities = df["embedding"].apply(lambda x: np.dot(x, query_vec) / (np.linalg.norm(x) * np.linalg.norm(query_vec)))

    # Get top_k most similar entries
    top_indices = similarities.nlargest(top_k).index

    return df.loc[top_indices][["qa_text", "Emp_name", "Dept_Name", "Access_Name"]]


In [24]:
# Example 1
print(answer_query("Which department does Amar Gupta belong to?", final_df))

# Example 2
print(answer_query("What access does Kajal have?", final_df))

# Example 3
print(answer_query("List employees in the Accounts and Finance department.", final_df))


                                             qa_text    Emp_name  \
1  Amar Gupta works in the Accounts and Finance d...  Amar Gupta   

              Dept_Name                                        Access_Name  
1  Accounts and Finance  [Teams, Outlook_Internal, Outlook_External, Ou...  
                                             qa_text         Emp_name  \
4  Kajal Prajapati works in the Accounts and Fina...  Kajal Prajapati   

              Dept_Name                                        Access_Name  
4  Accounts and Finance  [Teams, Outlook_Internal, Outlook_External, Ou...  
                                             qa_text    Emp_name  \
1  Amar Gupta works in the Accounts and Finance d...  Amar Gupta   

              Dept_Name                                        Access_Name  
1  Accounts and Finance  [Teams, Outlook_Internal, Outlook_External, Ou...  


In [13]:
qa_data = []

for _, row in final_df.iterrows():
    emp = row['Emp_name']
    dept_id = row['Dept_ID']
    dept_name = row['Dept_Name']
    access_ids = ", ".join(map(str, row['Access_ID']))
    access_names = ", ".join(row['Access_Name'])

    qa_data.extend([
        {"question": f"What department does {emp} belong to?", "answer": dept_name},
        {"question": f"What is the department ID of {emp}?", "answer": str(dept_id)},
        {"question": f"What accesses does {emp} have?", "answer": access_names},
        {"question": f"What are the access IDs assigned to {emp}?", "answer": access_ids},
        {"question": f"List the access names for {emp}", "answer": access_names},
        {"question": f"List the access IDs for {emp}", "answer": access_ids},
    ])

# Save as CSV
qa_df = pd.DataFrame(qa_data)
qa_df.to_csv("qa_dataset.csv", index=False)


In [17]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Use formatted text
context = ""
for _, row in final_df.iterrows():
    context += f"{row['Emp_name']} works in {row['Dept_Name']}. They have access to {', '.join(row['Access_Name'])}.\n"

# Ask a question
result = qa_model(question="What department does Amar Gupta belong to?", context=context)
print(result['answer'])

Accounts and Finance
